In [1]:
import numpy as np
import pandas as pd
import holoviews as hv
import panel as pn
import folium
import json
import matplotlib.cm as cm  # Import colormap library
import geopandas as gpd
import branca.colormap as cm
from colorcet import bmy

pn.extension('tabulator', template='fast')
import hvplot.pandas

## Create intro

In [2]:
introduction = pn.pane.Markdown("""
<h1>GeoPostal Insight</h1>
UPU Innovation Challenge 2024
""", width=600)

panel_logo = pn.pane.PNG(
    'https://panel.holoviz.org/_static/logo_stacked.png',
    link_url='https://panel.holoviz.org', height=95, align='center'
)

event_logo = pn.pane.PNG(
    'https://upload.wikimedia.org/wikipedia/commons/2/2d/Universal_Postal_Union_logo.svg',
    link_url='https://dribdat.hackathon.post/event/3', height=100, align='center'
)

itu_logo = pn.pane.PNG(
    'https://upload.wikimedia.org/wikipedia/commons/e/e1/International_Telecommunication_Union_logo.svg',
    link_url='https://dribdat.hackathon.post/event/3', height=100, align='center'
)

intro = pn.Row(
    event_logo,
    itu_logo,
    introduction,
    pn.layout.HSpacer(),
    panel_logo,
    sizing_mode='stretch_width'
)

intro

SyntaxError: invalid syntax (2743166477.py, line 18)

### Load and cache data

In [ ]:
from holoviews.element.tiles import lon_lat_to_easting_northing

@pn.cache
def load_data():
    df = pd.read_csv('data/brazil_geodata_h3_res4.csv')
    df['x'], df['y'] = lon_lat_to_easting_northing(df['X'], df['Y'])
    return df

df = load_data()

df.tail()

### Set up linked selections

In [ ]:
ls = hv.link_selections.instance()

def clear_selections(event):
    ls.selection_expr = None

clear_button = pn.widgets.Button(name='Clear', align='center')

clear_button.param.watch(clear_selections, 'clicks');

total_area = df.road_length_km.sum()

def count(data):
    selected_area  = np.sum(data['road_length_km'])
    selected_percentage = selected_area / total_area * 100
    return f'## Roads: {len(data)} | Selected: {selected_area:.0f} km ({selected_percentage:.1f}%)</font>'

pn.Row(
    pn.pane.Markdown(pn.bind(count, ls.selection_param(df)), align='center', width=600),
    clear_button
).servable(area='header', title='GeoPostal Insight')

In [ ]:
instruction = pn.pane.Markdown("""
💡 This dashboard visualizes postal offices, allows exploring the relationships between regional characteristics such as transport and telco infrastructures, complementary public services. Use Box-select on each plot to subselect and hit the "Clear" button to reset. Here you can ask prompts about our model, for example:
<ul><li>Create a report about the state of postal sector in Manaus region of Brazil.</li>
<li>What is the Integrated Index for Postal Development?</li></ul>
""", width=700)

# Todo: replace dynamically with output of prompt
hugging_pane = pn.pane.HTML("""
<iframe src="https://hf.co/chat/assistant/6659aa1af82f39d447cd13f3" width="100%" height="460" frameborder="0"></iframe>
""", width=700)

## Rich mapping

In [ ]:
# Load the GeoJSON data
geojson_data = "data/brazil_geodata_h3_res4.geojson"
gdf = gpd.read_file(geojson_data)

# Calculate the population count deciles for color mapping
population_counts = gdf['population_count']
population_deciles = pd.qcut(population_counts, 10, labels=False, duplicates='drop')

# Create a dictionary to map each population count to its decile
pop_count_to_decile = dict(zip(population_counts, population_deciles))

# Create a linear colormap
colormap = cm.linear.YlOrRd_09.scale(0, 9)
colormap = colormap.to_step(10)  # Use 10 steps for the deciles

# Define a function to style the features
def style_function(feature):
    population_count = feature['properties']['population_count']
    decile = pop_count_to_decile[population_count]
    return {
        'fillOpacity': 0.7,
        'weight': 0,
        'fillColor': colormap(decile)
    }

# Create the map
m = folium.Map(location=[-9.29, -51.81], zoom_start=3, tiles="cartodb positron")
folium_pane = pn.pane.plot.Folium(m, height=460)
folium.Marker(
    [-9.29, -51.81], popup="<i>Brazil</i>", tooltip="Work in progress!"
).add_to(m)

# Add the GeoJSON data to the map with the style function
folium.GeoJson(
    geojson_data,
    style_function=style_function,
    tooltip=folium.GeoJsonTooltip(fields=['population_count']),
    name="GeoPostal Impact"
).add_to(m)

# Add a layer control
folium.LayerControl().add_to(m)

# Set the pane object to the map
folium_pane.object = m

# Create the dashboard layout
h3ai = pn.Row(
    folium_pane,
    hugging_pane
)
h3ai

### Create plots

In [ ]:
df['post_offices_count_int'] = df['post_offices_count'].fillna(0).astype(int)
df['postal_bank_count_int'] = df['postal_bank_count'].fillna(0).astype(int)

geo = df.hvplot.points(
    'x', 'y', rasterize=True, tools=['hover'], tiles='OSM', cmap=bmy, logz=False, colorbar=True,
    xaxis=None, yaxis=False, min_height=400, responsive=True
).opts('Tiles', alpha=0.8)

scatter1 = df.hvplot.scatter(
    'post_offices_count', 'population_count', 
    xlabel='Post offices', ylabel='Population', color='postal_bank_count_int', responsive=True, min_height=400
)

scatter2 = df.hvplot.scatter(
    'road_length_km', 'post_offices_count', 
    xlabel='Road length', ylabel='Post offices', responsive=True, min_height=400
)

scatter3 = df.hvplot.scatter(
    'cell_towers_count', 'post_offices_count', 
    xlabel='Cell towers', ylabel='Post offices', responsive=True, min_height=400
)

hist1 = df.hvplot.hist(
    'post_offices_count_int', xlabel='Number of Post offices',
    fontscale=1.2, responsive=True, min_height=350, fill_color='#85c1e9'
)


hist2 = df.hvplot.hist(
    'postal_bank_count_int', xlabel='Number of Postal banks',
    fontscale=1.2, responsive=True, min_height=350, fill_color='#f1948a'
)

plots = pn.pane.HoloViews(ls(geo + scatter1 + hist1 + hist2 + scatter2 + scatter3).cols(2).opts(sizing_mode='stretch_both'))
plots

## Dashboard content


In [ ]:
pn.Column(intro, instruction, h3ai, plots, sizing_mode='stretch_both').servable();

